In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

mydriver= webdriver.Chrome() 
#Add category and category link
category="Add category name"
mydriver.get(r"add category link")

In [ ]:
while True:
    try:
        # Wait for the "Next" button to be clickable
        next_button = WebDriverWait(mydriver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='pager-button']"))
        )
        
        # Scroll the button into view
        actions = ActionChains(mydriver)
        actions.move_to_element(next_button).perform()
        
        # Add a small delay
        time.sleep(1)
        
        # Click the button using JavaScript
        mydriver.execute_script("arguments[0].click();", next_button)
        
        # Wait for the page to load
        time.sleep(2)
        
    except ElementClickInterceptedException:
        print("Element is still obstructed. Retrying...")
        continue  # Retry clicking the button
        
    except (NoSuchElementException, TimeoutException):
        print("No more pages to load or 'Next' button not found.")
        break  # Exit the loop if the "Next" button is not found


In [ ]:
items_links=[]
#page items:
my_elems = mydriver.find_elements("xpath","//div[@class='product-item card']")

for i in range(len(my_elems)):
        items_links.append(my_elems[i].find_element("xpath","a[@class='product-item-link']").get_attribute("href"))
print(len(items_links))
items_links

In [ ]:
# Create an empty DataFrame
df = pd.DataFrame(columns=["item title", "original price", "price on sale", "rating", "color options", "size options", "description", "Age Group", "Garment Length", "Presentation Product Type", "Sleeve Length", "Composition", "category"])
item_id=0
while(item_id<len(items_links)):
    try:
        print(f"******************************item {item_id}***************************")
        link=items_links[item_id]
        #go to link
        mydriver.get(link)

        # Wait until the document is ready
        time.sleep(2)

        data=[]

        #product_title
        data.append(mydriver.find_element("xpath", "//h6[@class='pdp-product__title']").text)

        #price before
        try:
            data.append(mydriver.find_element("xpath","//div[@class='pdp-product__prices']//span[@class='dropin-price dropin-price--small dropin-price--bold dropin-price--strikethrough']").text)
        except NoSuchElementException:
            data.append(np.nan)

        #price after
        data.append(mydriver.find_element("xpath","//div[@class='pdp-product__prices']//span[@class='dropin-price dropin-price--default dropin-price--small dropin-price--bold']").text)

        #rating:a class="pdp-product__ratings--count"
        try:
            mydriver.find_element("xpath","//a[@class='pdp-product__ratings--count']").click()
        except NoSuchElementException:
            data.append(np.nan)
        else:
            data.append(mydriver.find_element("xpath","//span[@class='pdp-product__ratings--average']").text)
            mydriver.find_elements("xpath","//div[@class='modal-header']/button[@class='modal-close']/span[@class='icon icon-close']/img[@src='/icons/close.svg']")[-1].click()

        #available colors:
        colors=[]
        my_elems = mydriver.find_elements("xpath", "//div[@class='pdp-swatches-list-container']/div[@class='pdp-swatches-refs']/a")
        for i in range(len(my_elems)):
            colors.append(my_elems[i].get_attribute('data-swatch-title'))
        data.append(colors)

        #available sizes:
        sizes=[]
        my_elems = mydriver.find_elements("xpath", "//div[@class='pdp-swatches__options']//label[@class='dropin-text-swatch__label']")
        for i in range(len(my_elems)):
            sizes.append(my_elems[i].text)
        data.append(sizes)

        #description section
        mydriver.find_element("xpath", "//h3[@class='pdp-product__description--title']").click()
        #description content
        data.append(mydriver.find_element("xpath", "//div[@class='pdp-product__description--content']").text)
        #description attributes:
        attributes={"Age Group":np.nan,"Garment Length":np.nan,"Presentation Product Type":np.nan,"Sleeve Length":np.nan,"Composition":np.nan}
        my_elems = mydriver.find_elements("xpath", "//div[@class='pdp-product-description__attributes']/ul[@class='pdp-product-description__attribute']/li")
        for i in range(len(my_elems)):
            attribute=my_elems[i].find_element(By.XPATH,".//div").text
            if attribute in attributes.keys():
                attributes[attribute]=my_elems[i].find_element(By.XPATH,".//span").text
        data=data+list(attributes.values())
        data.append(category)
        # Add data to DataFrame
        df.loc[len(df)] = data
        print(data)
        item_id+=1
    except:
        print(f"try again item {item_id}")

# Save the DataFrame to a CSV file (optional)
df.to_csv(category+".csv", index=False)